In [1]:

constants = {

    # paramater for string
    "Ns": 100,              # number of points to discretize the string
    "ls": 0.64,             # length of the string
    "rho_s": 1.0,           # density of the string
    "T": 1.0,               # the tension of the string

    # parameter for plucking force
    "t1": 0.1,              # the time when the plucking force is maximum
    "t2": 0.2,              # the time when the plucking force is released
    "delta_s": 0.1,         # space extent of the plucking force
    
    # parameter for plate
    "D1": 850.0,
    "D2": 50.0,
    "D3": 75.0,
    "D4": 200.0,
    "rho_p": 350,
    "eta_p": 0.005,
    "R_p": 7,
    'a': 0.0029,
    "plate vertical axis": 1,
    "plate thd": -0.04645,
    "plate x axis": 0,
    "plate x min": -0.075,
    "plate x max": 0.075,
    "plate y axis": 2,
    "plate y min": 0.25,
    "plate y max": 0.40,
    "gamma_0_ref_dir": False,
    "gamma_f_ref_dir": False,

    # parameter for air 


    # coupling parameters
    "x0": 0, "y0": 0.1, # the position of the interaction point between the string and the plate

}

variables = {
    'M_sh': None,
    'D_sh': None,
    'f_sh': None,
    'M_qh': None,

    'M_ph': None,
    
}


In [2]:
from models import String
string = String(constants)
variables['M_sh'] = string.M_sh
variables['D_h'] = string.D_h
variables['f_sh'] = string.get_f_sh
variables['M_qh'] = string.M_qh

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
import open3d as o3d
import numpy as np
mesh = o3d.io.read_triangle_mesh("sound_board.stl")

vertices = np.asanyarray(mesh.vertices)
triangles = np.asanyarray(mesh.triangles)

vertices, inverse_indices = np.unique(vertices, axis=0, return_inverse=True)
triangles = inverse_indices[triangles]

from models import SoundBoard
soundBoard = SoundBoard(vertices, triangles, constants)

variables['M_ph'] = soundBoard.M_ph


SoundBoard Info:
# of vertices: 3508
# of edges: 10200
# of centriods: 6692
# of total points: 20400
# of edges on gamma_f: 112
# of edges on gamma_0: 212


In [4]:
np.max(vertices, 0)

array([0.18651307, 0.05432427, 0.48692846])

In [5]:
np.min(vertices, 0)

array([-0.18595666, -0.04650379,  0.00143154])

In [6]:
from models import Air

# air = Air(constants)
# variables['M_ah'] = air.M_ah
# variables['D_h'] = air.D_h

In [7]:
variables

{'M_sh': <99x99 sparse matrix of type '<class 'numpy.float64'>'
 	with 99 stored elements in COOrdinate format>,
 'D_sh': None,
 'f_sh': <bound method String.get_f_sh of <models.String object at 0x747a2833e350>>,
 'M_qh': <100x100 sparse matrix of type '<class 'numpy.float64'>'
 	with 396 stored elements in COOrdinate format>,
 'M_ph': <20400x20400 sparse matrix of type '<class 'numpy.float64'>'
 	with 242208 stored elements in Dictionary Of Keys format>,
 'D_h': <99x100 sparse matrix of type '<class 'numpy.float64'>'
 	with 198 stored elements in COOrdinate format>}